In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from tape import ProteinBertModel, TAPETokenizer, ProteinBertForMaskedLM
# model = ProteinBertModel.from_pretrained('bert-base', force_download=True)
model = ProteinBertForMaskedLM.from_pretrained('bert-base')#, force_download=True)
tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model
model = model.to("cpu")
model

/projects/ashehu/akabir4/venvs/hopper_tape_rao/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ProteinBertForMaskedLM(
  (bert): ProteinBertModel(
    (embeddings): ProteinBertEmbeddings(
      (word_embeddings): Embedding(30, 768, padding_idx=0)
      (position_embeddings): Embedding(8192, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ProteinBertEncoder(
      (layer): ModuleList(
        (0): ProteinBertLayer(
          (attention): ProteinBertAttention(
            (self): ProteinBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ProteinBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm()
              (dropout): D

In [3]:
model.mlm.decoder.weight

Parameter containing:
tensor([[ 0.0079, -0.0353, -0.0143,  ...,  0.0406, -0.0008,  0.0108],
        [ 0.0398, -0.0075, -0.0351,  ...,  0.0365,  0.0097,  0.0137],
        [-0.0168, -0.0186, -0.0212,  ..., -0.0174,  0.0334, -0.0053],
        ...,
        [-0.0386, -0.0032, -0.0156,  ...,  0.0421, -0.0131,  0.0138],
        [-0.0392, -0.0253, -0.0334,  ..., -0.0235,  0.0122, -0.0710],
        [ 0.0128, -0.0218, -0.0280,  ...,  0.0126, -0.0004,  0.0260]],
       requires_grad=True)

In [5]:
tokenizer.vocab

OrderedDict([('<pad>', 0),
             ('<mask>', 1),
             ('<cls>', 2),
             ('<sep>', 3),
             ('<unk>', 4),
             ('A', 5),
             ('B', 6),
             ('C', 7),
             ('D', 8),
             ('E', 9),
             ('F', 10),
             ('G', 11),
             ('H', 12),
             ('I', 13),
             ('K', 14),
             ('L', 15),
             ('M', 16),
             ('N', 17),
             ('O', 18),
             ('P', 19),
             ('Q', 20),
             ('R', 21),
             ('S', 22),
             ('T', 23),
             ('U', 24),
             ('V', 25),
             ('W', 26),
             ('X', 27),
             ('Y', 28),
             ('Z', 29)])

In [12]:
import utils.pickle_utils as pickle_utils
logits_1 = pickle_utils.load_pickle(home_dir+f"models/tape_rao/outputs/protbert/lm_outputs/A000006_2_271.pkl") 
logits_2 = pickle_utils.load_pickle(home_dir+f"models/tape_rao/outputs/protbert/lm_outputs/A000006_2_62.pkl") 

# now see, logits in masked position are different, 
print(logits_1[270])
print(logits_2[270])

[-14.572329   -11.0084915  -12.411443    -9.077562   -14.077164
   1.4255943  -12.878991     0.9913696    1.451469     1.3550032
   2.0718853    1.5725293    1.2201787    2.0912328    1.5908126
   2.3223362    0.8785827    1.8224728  -12.962563     1.5508606
   1.4145026    1.4866153    5.767225     1.8694699   -9.500174
   1.806958     0.81860864  -5.78228      1.7986705  -12.370768  ]
[-14.41481    -10.875704   -12.323403    -8.911601   -13.955445
   1.4228826  -12.721293     1.0631462    1.420688     1.2446738
   2.1930194    1.5513582    1.2077374    2.156792     1.5138537
   2.4022515    0.9481643    1.7951783  -12.835764     1.4154377
   1.3266288    1.465413     5.7771254    1.8574655   -9.422638
   1.8540447    0.81553805  -5.622466     1.8164994  -12.252094  ]


In [21]:
import torch
import numpy as np
seq = list('GCA')
mut_pos_zero_idxd = 1
seq[mut_pos_zero_idxd] = "<mask>"
print(seq)
token_ids = torch.tensor(np.array([tokenizer.encode(seq)]))
print(token_ids)
with torch.no_grad():
    output = model(token_ids)[0][0].detach().numpy()
print(output.shape)
output

['G', '<mask>', 'A']
tensor([[ 2, 11,  1,  5,  3]])
(5, 30)


array([[  0.9393923 ,   1.4764738 ,  17.069212  ,   4.093584  ,
          2.2064114 ,   3.540858  ,   1.4473366 ,   2.1282196 ,
          1.6636357 ,   3.559037  ,   0.75567114,   1.4647713 ,
          3.0634506 ,  -0.91880214,  -1.1657937 ,   3.1354527 ,
          3.758852  ,   4.13353   ,   2.2984781 ,  -1.1430006 ,
          1.0028296 ,  -0.73765385,   1.8968856 ,   2.0362754 ,
          1.1207702 ,   1.3910527 ,  -0.8755777 ,   0.6121899 ,
          1.4712923 ,   1.8130984 ],
       [ -9.174221  ,  -7.324638  ,  -7.451663  ,  -2.8247697 ,
         -8.93935   ,   2.8364978 ,  -8.080409  ,  -2.5297391 ,
          4.2169266 ,   2.5058782 ,  -0.0582124 ,   8.556776  ,
          1.5619744 ,  -0.62475365,   3.1155698 ,   0.312629  ,
          0.7278353 ,   3.1459618 ,  -8.427874  ,   3.038825  ,
          2.153031  ,   3.0776536 ,   2.90188   ,   2.3127396 ,
         -6.1784453 ,   0.7753676 ,  -2.248705  ,  -1.1377429 ,
         -0.21283606,  -8.4147415 ],
       [-13.875187  ,  -9.9652

In [2]:
from tape import UniRepForLM, TAPETokenizer, UniRepModel
tokenizer = TAPETokenizer(vocab='unirep')

cache_dir=home_dir+"models/tape_rao/cache/unirep"
model = UniRepForLM.from_pretrained('babbler-1900', cache_dir=cache_dir) # force_download=True
# model = UniRepModel.from_pretrained('babbler-1900', force_download=True, cache_dir=cache_dir)
model = model.to("cpu")
model.eval()

/projects/ashehu/akabir4/venvs/hopper_tape_rao/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 72879855/72879855 [00:01<00:00, 42691361.66B/s]


UniRepForLM(
  (unirep): UniRepModel(
    (embed_matrix): Embedding(26, 10)
    (encoder): mLSTM(
      (mlstm_cell): mLSTMCell(
        (wmx): Linear(in_features=10, out_features=1900, bias=False)
        (wmh): Linear(in_features=1900, out_features=1900, bias=False)
        (wx): Linear(in_features=10, out_features=7600, bias=False)
        (wh): Linear(in_features=1900, out_features=7600, bias=True)
      )
    )
  )
  (feedforward): Linear(in_features=1900, out_features=25, bias=True)
)

In [3]:
print(model.feedforward.weight)

Parameter containing:
tensor([[-0.0335,  0.0224, -0.0772,  ...,  0.0020,  0.0077, -0.0977],
        [ 0.1135,  0.0177, -0.1351,  ..., -0.0046, -0.0181, -0.0206],
        [-0.1942, -0.0078, -0.1548,  ..., -0.0168, -0.0323, -0.0569],
        ...,
        [-0.1097,  0.0126,  0.0210,  ...,  0.0075,  0.0143,  0.0834],
        [-0.1586,  0.0021,  0.0858,  ...,  0.0097,  0.0323,  0.0714],
        [-0.4907,  0.0552,  0.0380,  ..., -0.0711, -0.0339, -0.0136]],
       requires_grad=True)


In [65]:
import torch
import numpy as np
print(tokenizer.vocab)
seq = 'GCTVED'

token_ids = torch.tensor(np.array([tokenizer.encode(seq)]))
print(token_ids)
with torch.no_grad():
    output = model(token_ids)
output

OrderedDict([('<pad>', 0), ('M', 1), ('R', 2), ('H', 3), ('K', 4), ('D', 5), ('E', 6), ('S', 7), ('T', 8), ('N', 9), ('Q', 10), ('C', 11), ('U', 12), ('G', 13), ('P', 14), ('A', 15), ('V', 16), ('I', 17), ('F', 18), ('Y', 19), ('W', 20), ('L', 21), ('O', 22), ('X', 23), ('Z', 23), ('B', 23), ('J', 23), ('<cls>', 24), ('<sep>', 25)])
tensor([[24, 13, 11,  8, 16,  6,  5, 25]])


(tensor([[[  4.7122,  -0.9693,  -1.7770,  -0.9777,  -1.5403,  -1.2578,  -1.0279,
            -1.2994,  -1.1935,  -1.3617,  -2.1368, -10.2135,  -0.9097,  -0.9102,
            -0.7804,  -0.9665,  -1.1355,  -1.1997,  -1.4462,  -2.5935,  -0.3936,
           -10.4988, -10.9553, -10.3032,  -6.3302],
          [ -2.1055,  -0.8174,  -2.0496,  -0.8736,  -1.3527,  -0.9619,  -0.6972,
            -1.0954,  -1.4274,  -1.6354,  -2.1289, -12.5411,  -0.4415,  -0.9722,
            -0.6240,  -0.8490,  -1.0964,  -1.5946,  -2.0409,  -2.3955,  -0.4821,
           -13.4826, -13.4685, -13.4728,  -4.6399],
          [ -2.4944,  -1.1877,  -2.1952,  -1.7645,  -1.5720,  -1.6680,  -0.8755,
            -1.5376,  -1.8708,  -2.0873,  -1.7326, -13.1058,  -0.7995,  -1.2540,
            -1.1414,  -1.3046,  -1.6317,  -1.5870,  -2.0093,  -2.4680,  -0.9934,
           -13.9631, -14.0157, -13.8544,  -4.1981],
          [ -2.2567,  -1.3499,  -2.1782,  -1.6112,  -1.5732,  -1.5904,  -0.8524,
            -1.1852,  -1.6349,  -1